In [1]:
# 경고끄기
import warnings
warnings.filterwarnings('ignore')
import xgboost as xgb
xgb.set_config(verbosity=0)

XGBoost 모델 만들기
  - 붓꽃데이터

In [2]:
import pandas as pd
import numpy as np
from sklearn import datasets
iris = datasets.load_iris()

In [8]:
iris['data'][:3], iris['feature_names'], iris['target'][:3]

(array([[5.1, 3.5, 1.4, 0.2],
        [4.9, 3. , 1.4, 0.2],
        [4.7, 3.2, 1.3, 0.2]]),
 ['sepal length (cm)',
  'sepal width (cm)',
  'petal length (cm)',
  'petal width (cm)'],
 array([0, 0, 0]))

In [ ]:
np.hstack( (iris['data'], iris['target'].reshape(-1,1) ) )

In [28]:
df = pd.DataFrame(data=np.hstack( (iris['data'], iris['target'].reshape(-1,1) ) ),columns=(iris['feature_names']+['target']) )
df.head(1)

,sepal length (cm),sepal width (cm),petal length (cm),petal width (cm),target
0,5.1,3.5,1.4,0.2,0.0


In [29]:
# 훈련 테스트 데이터
from sklearn.model_selection import train_test_split
X_train,X_test,y_train,y_test =  train_test_split(iris['data'], iris['target'],random_state=2)

XGBoost 분류 모델

In [30]:
from xgboost import XGBClassifier
from sklearn.metrics import accuracy_score

In [32]:
# gbtree 기본값
# objective 손실함수
# multi:softprob(클래스별 예측확률) 다중 클래스 분류문제 multi:softmax(클래스 레이블)
xgb = XGBClassifier(booster = 'gbtree', objective='multi:softprob',
                    max_depth = 6, learning_rate = 0.1, n_estimators=100, n_jobs=-1
)
xgb.fit(X_train,y_train)
y_pred = xgb.predict(X_test)
score = accuracy_score(y_test,y_pred)
print(f"점수 : {score}")

점수 : 0.9736842105263158


XGBoost 모델을 빠르고 효율적으로 데이터 처리

In [38]:
import xgboost as xgb
dtrain = xgb.DMatrix(X_train,y_train) # 데이터 행렬구조  학습데이터와 테스트 데이터를 표현
dtest = xgb.DMatrix(X_test[:5])

param = {'objective':'multi:softprob','num_class':3}
bstr =  xgb.train(param,dtrain,10)  # 10개의 트리를 학습
bstr.predict(dtest)

array([[0.9486482 , 0.02711029, 0.02424142],
       [0.9486482 , 0.02711029, 0.02424142],
       [0.02841366, 0.05416913, 0.9174172 ],
       [0.9486482 , 0.02711029, 0.02424142],
       [0.9486482 , 0.02711029, 0.02424142]], dtype=float32)

In [39]:
param = {'objective':'multi:softmax','num_class':3}
bstr =  xgb.train(param,dtrain,10)  # 10개의 트리를 학습
bstr.predict(dtest)

array([0., 0., 2., 0., 0.], dtype=float32)

당뇨병 데이터셋
  - XGBoost 회귀모델(교차 검증)

In [40]:
X,y =  datasets.load_diabetes(return_X_y=True)

In [43]:
from sklearn.model_selection import cross_val_score
from xgboost import XGBRegressor
xgb = XGBRegressor(booster = 'gbtree',objective='reg:squarederror',max_depth=6,learning_rate=0.1,n_estimators=100,n_jobs=-1)
scores = cross_val_score(xgb,X,y,scoring = 'neg_mean_squared_error',cv=5)
# 제곱근
rmse = np.sqrt(-scores)
# rmse 출력
print(f"RMSE : {np.round(rmse,3)}")
print(f"RMSE 평균 : {np.mean(rmse)}")

RMSE : [59.397 60.322 69.036 63.211 66.953]
RMSE 평균 : 63.78393977997189
